In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV

In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))

    return result

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

In [4]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(".\\source\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
x_train, x_test, y_train, y_test = load_data(test_size=0.25)
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(1080, 360)
Features extracted: 180


In [6]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [28]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,250,), learning_rate='adaptive', max_iter=800)
model.fit(x_train_scaled, y_train)
y_pred_scaled = model.predict(x_test_scaled)
accuracy_scaled = accuracy_score(y_true=y_test, y_pred=y_pred_scaled)
print("Accuracy: {:.2f}%".format(accuracy_scaled * 100))

Accuracy: 63.06%


In [33]:
y_pred_scaled = model.predict(x_test_scaled)
accuracy_scaled = accuracy_score(y_true=y_test, y_pred=y_pred_scaled)
print("Accuracy: {:.2f}%".format(accuracy_scaled * 100))

Accuracy: 63.06%


In [35]:
from joblib import dump

dump(model, 'model.joblib')


['model.joblib']

In [37]:
from joblib import load

loaded_model = load('model.joblib')
features = extract_feature("source/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)
scaler = StandardScaler()
scaled_features = scaler.transform(features.reshape(1, -1))
predicted_emotion = model.predict(scaled_features)

print("Predicted emotion:", predicted_emotion)

FileNotFoundError: [Errno 2] No such file or directory: 'model.joblib'